# Spectral LDA on Spark

## Summary

There has been tremendous excitement about machine learning and artificial intelligence over the last few years. We are now able to do automated classification of images, carry out speech recognition, and process textual information efficiently. However, all these tasks fall under what we call supervised learning, where the training data provides label information. For instance,  in image recognition, the categories of images are pre-specified.  What if such information about the categories is absent? Can we have an algorithm which automatically discovers the categories? This is known as unsupervised learning and experts agree that it is one of the hardest problems in machine learning.



Latent variable models are proven to be versatile and have applications in almost every domain, including natural language processing, social network analysis and computer vision. Training large scale latent variable models are challenging due to the non-convexity and intractability of the likelihood function. Today we will introduce a new spectral framework to train **Latent Dirichlet Allocation** (a.k.a. LDA) Model with the application of discovering hidden topics for documents.



## What is LDA?

It has been postulated that humans mostly learn in an unsupervised manner   gathering “common-sense” or “general-purpose” knowledge. For instance, let us look at this sample New York times article: the headline reads “At Florida State, Football Clouds Justice” What comes to your mind? Almost immediately, you would know that the article is talking about Crime, Sports and probably Education as well. But how did you arrive at this conclusion?  through years of accumulated knowledge and learning.  Now the big question is can we design machines with such capabilities?

![Fig1](figures/NewYorkArticle-Example-with-Model00.svg)
<center><bold>Figure 1 An example of New York Times article.</bold></center>



The general problem of unsupervised learning is enormously challenging. So let us start with some simple concrete goals. Going back to the New York times article that Furong introduced, can we design an algorithm that automatically discovers the categories of topics that the article covers. To do this, let us assume we are given access to a large corpus of  news articles for learning. 
Let us start with a very simple algorithm: let us just count the words in each article. By doing so, we are indeed ignoring a whole lot of information: the order in which words occur, the paragraphs they are organized into, and so on. But what kind of information can be gleaned by just looking at frequency of word occurrences? It turns out quite a bit: We will see how these word counts can help us discover topic categories efficiently. 


The bag-of-words topic modeling is a simplifying representation where a document is modeled as a “bag of its words”, disregarding the word order.




### Bag of Words Model and Graphical Model Representation

![Fig2](figures/TopicModelPlate.jpg)
<center>Figure 2 Latent Dirichlet Allocation (LDA) plate representation.</center>



Let us now introduce a popular model known as latent Dirichlet allocation (LDA). The LDA model puts a Dirichlet distributed prior $\alpha=[\alpha_1,\alpha_2,…,\alpha_k]$ on the topics of the entire documents, and assumes that the documents are generated in the following manner:



![Fig3](figures/NewYorkArticle-Example-with-Model0.svg)
<center>Figure 3 LDA: Sample the topic propotion distribution from Dirichlet prior for current article.</center>


1. For each document $i$, topic of the document is categorical. We sample a simplex $\theta_i\in \Delta^k$ from the Dirichlet prior. This simplex $\theta_i$ is the categorical distribution of the topic proportions. Refer to Figure 3.
2. For each word $j$ in document $i$, the generative model first decides the topic of word $j$, say $z_j$, based on the topic distribution $\theta_i$; then the generative model decides which word to put according to the topic word probability $\Pr(w_j|z_j)$, i.e., probability of any word in the vocabulary occurs at position $j$ under topic $z_j$. Note that $z_j$ can be $\{1,2,..,k\}$. Refer to Figure 4.


![Fig4](figures/NewYorkArticle-Example-with-Model.svg)
<center>Figure 4 LDA: Generate words after determing the topic.</center>





We illustrate an example of graphical model representation of LDA for three hidden topics: yellow, pink and cyan topic in Figure 4. The generative model assume a dirichlet prior on these three hidden topics. Among the $M$ documents, we show the generative model for a single document where 5 words are displayed. Based on the topic proportions sampled from the dirichlet prior, we decide to focus on a pink topic, and generated the word based on the topic-word probability table. Here we see word “campus” as campus is a high frequent word under the pink topic. 


![Fig5](figures/topicmodel-new.svg)
<center>Figure 5 LDA: Graphical model representation.</center>


Our goal of learning LDA is to find the hidden topic word table $A$ that drives the observed words, where $A_{ij} = \Pr(\text{word is }i| \text{topic is }j)$.

### Learning LDA through Tensor Decomposition
The challenges for efficient and accurate learning of LDA topic modeling are as follows: 

1.	English vocabulary contains hundreds of thousands individual words.
2.	Number of documents are large.
3.	Topics are hidden.
4.	MCMC such as Gibbs sampling is slow and requires exponential mixing time; variational inference is faster but optimizes over the lower bound of likelihood, and is susceptible to spurious local optima. 

We propose spectral methods (matrix and tensor decomposition) as the solution. It is faster than Gibbs sampling and variational inference; it is scalable; and it has theoretical guarantees for learning the correct model (the topic-word matrix and the Dirichlet prior). 


Here is how our framework works. We have designed a new approach for learning topic models. This is based on the idea of computing moments or aggregates. Now let us take the entire collection of news articles and the word count vectors associated with them. if we take the average of all these vectors, we get an estimate of the average word frequencies over the entire document collection. If you do the math, it turns out that this average frequency is given by a linear combination of the columns of the topic-word table introduced earlier. Note that each column represents one topic: it tells us how likely different words are under that topic.


![Fig61](figures/Rubik_Cube_1d_decomposition_long_labeled.svg)
<center>Figure 6 First order data moments of LDA is the average frequency vector. The average frequency vector is equal to a linear combination of the columns of the topic-word table.</center>


The average frequency vector is not enough to learn the topic-word table, and we need to calculate co-occurrences of pairs of words. That is, we take every pair of words and ask how many times on average they occur in the document collection. A modified version of this word-pair frequency matrix can be expressed as sum of rank-1 components where each rank-1 component corresponds to one column of the topic-word table. For a matrix, the singular value decomposition gives us rank-1 components which are orthogonal to one another. However, the columns of topic-word table are not necessarily orthogonal in general.  So there is simply not enough information to uniquely identify the topic-word table from pair wise frequency information. 

![Fig62](figures/Rubik_Cube_2d_decomposition_long_labeled.svg)
<center>Figure 7 Second order data moments of LDA is the modified co-occurence frequency of the pairs of words. The modified co-occurence frequency of the pairs of words is equal to linear combination of rank-1 components where each rank-1 component corresponds to one column of the topic-word table.</center>






We need to look at relationships among triplets of words. This is where we need the concept of tensors. Tensors are generalizations of matrices. You can visualize third order tensor as a cube. In each entry of the cube you can store information about how many times a specific triplet of words co-occurred on average over the document collection.

It turns out that for LDA model, modified version of the third order tensor that records triplet word co-occurrences can be expressed as a sum of rank-1 components, where each component corresponds to a column of the topic-word table. This expression is an extension of the matrix decomposition.By finding the decomposition of this tensor, we can uniquely identify the topic-word table under some pretty mild conditions. Thus tensors are much more powerful than matrices. You can find more details about tensors and their properties in our technical papers. 

![Fig62](figures/Rubik_Cube_decomposition_long_labeled.svg)
<center>Figure 8 Third order data moments of LDA is the modified co-occurence frequency of the triplets of words. The modified co-occurence frequency of the triplets of words is equal to linear combination of rank-1 components where each rank-1 component corresponds to one column of the topic-word table.</center>







### Alternating Least Square Algorithm

We will describe a popular framework known as alternating least squares or ALS. Let us look at the rank-1 components that constitute the decomposition. Now each rank-1 component  is made up of three vectors: one along each of the three directions. What the ALS method does is it fix two of the directions, and it updates only the set of vectors along the third direction. This becomes a linear update that can be done efficiently. You can see this more clearly in the following figures.


#### Slice of the tensor share common comoponents


The input tensor has three directions given by $i_1$, $i_2$ and $i_3$. There are two rank-1 components: red and blue. The red rank-1 component is further made up of vectors along three directions: represented from dark to lighter shades of red, and similarly for the blue component. 
![Fig6](figures/ALS-Step-00.svg)
<center>Figure 9 ALS: Tensor decomposition form.</center>

Now let’s take a slice of this input tensor,  this is a matrix. This matrix has the nice property that it shares the same rank-1 components as the input tensor. As you can see in the figure, the rank-1 components of the matrix are again the red and blue set of vectors, scaled appropriately. Hence all tensor slices share the same red and blue component vectors. 

![Fig8](figures/ALS-Step-02.svg)
<center>Figure 10 ALS: A slice of the tensor shares same components as the tensor with appropriate scaling factors.</center>

![Fig9](figures/ALS-Step-03.svg)
<center>Figure 11 ALS: Multiply the scaling factors with vectors in the $i_2$ direction.</center>


#### Unfolded tensor has nice structures

So if we collect all the slices of the tensor together and place them side by side, we get a bigger matrix. We can then exploit this very nice structure of shared components to find the tensor decomposition. 


![Fig10](figures/ALS-Step-04-new.svg)
<center>Figure 12 ALS: Original tensor decomposition form.</center>

![Fig11](figures/ALS-Step-05-new.svg)
<center>Figure 13 ALS: Unfoled tensor decomposition form.</center>

The alternating least squares works as follows: we will update the values in the dark red and blue vectors while fixing the vectors with the smiley faces. As you can see this update  only requires  simple matrix inversion and multiplication steps. Similarly we can update the other vectors with lighter shades in other directions of the tensor



![Fig12](figures/ALS-Step-06-new.svg)
<center>Figure 14 ALS: Reformulated unfolded tensor decomposition form.</center>





![Fig13](figures/ALS-Step-07-new.svg)
<center>Figure 15 ALS: The algorithm requires simple matrix inversion and multiplications.</center>


Therefore, the alternating least squares method simpley requires a bunch of matrix inversion and multiplication steps. But done naively, this is still not practical. Matrix inversion is a cubic time algorithm, and this matrix is enormous: the larger dimension of this matrix is the square of the size of  English vocabulary!


### Dimensionality reduction
This is where we bring in another trick: dimensionality reduction. We will  ultimately run alternating least squares on a much smaller tensor. But how do we reduce the size and yet retain information? Linear algebra again comes to our rescue.  Here,  we revisit the modified pairwise word co-occurrence matrix, we saw earlier:

![Fig13](figures/pairsMatrix.svg)
<center>Figure 16 The modified pairwise word co-occurence matrix.</center>



By computing singular value decomposition of this matrix, we can get the required transformation to reduce dimension of the triplet-word tensor and yet preserve information. The dimensionality reduction is drastic: along each direction of the new tensor, the length  is just the number of topics in the model. This is much much smaller than the English vocabulary: we usually have a few hundred topics vs. hundreds of thousands of  possible words. Now This is great, but we  can reduce computation even further. We do not need to compute the exact singular value decomposition of the pairs matrix. Instead, we can do a much faster randomized scheme which is guaranteed to give approximate answers. You can refer to some excellent papers by Joel Tropp for details.


![Fig13](figures/dimensionality-reduction.svg)
<center>Figure 17 Dimensionality reduction.</center>


So the overall framework for tensor decomposition is as follows: first compute randomized SVD of the pairs matrix, and then use it to reduce the length of word count vectors in each document. After transformation, the new vectors have length equal to the number of topics. Now we compute the third order moment over this new collection of shorter vectors. We then  find the rank-1 components of this smaller tensor using alternating least squares algorithm. Finally, we project the output on to the word space through simple matrix multiplications to obtain the parameters of the topic model.



## To Use the API
We built the library at https://github.com/FurongHuang/spectrallda-tensorspark and published on Spark Packages https://spark-packages.org/package/FurongHuang/SpectralLDA-TensorSpark. As a simple example we could use it in Spark Shell. We start the shell by

```bash
spark-shell --packages FurongHuang:SpectralLDA-TensorSpark:1.0
```

We create a `TensorLDA` instance to look for the top 10 topics, supposing the prior parameter $\alpha_0=5.0$.

```scala
import edu.uci.eecs.spectralLDA.algorithm.TensorLDA

val lda = new TensorLDA(dimK = 10, alpha0 = 5.0)
```

For the collection of documents `docs`, it'd expect it of type `RDD[(Long, breeze.linalg.SparseVector[Double])]`, where each tuple consists of a `Long` document ID and the sparse word count vector of that document. To fit the model on `docs` and get the model parameters `alpha`, `beta`, simply do

```scala
val (beta, alpha, _, _, _) = lda.fit(docs)
```

For more details, please refer to the README of the GitHub repository.


## Visualization of New York Times

We run Spectral LDA on New York Times dataset, and intepretated the results in this page: http://newport.eecs.uci.edu/anandkumar/Lab/Lab_sub/NewYorkTimes3.html. 
This shows that we are able to successfully discover a few hot topics such as Economy, Education, Sports, Online Social Media and Crime Reports. 

Once we have learnt the parameters of model using spectral LDA, we can go back to the sample article  we initially introduced. When we run the inference step, we find that it successfully discovers that the article talks both about crime and  education.

![Fig13](figures/Topic_Article5.svg)
<center>Figure 18 The predicted topics for the sample new york times article.</center>


